# Interval computations - Labs

## Prerequisites

In [1]:
!pip install -q pyinterval

     |████████████████████████████████| 1.6MB 10.0MB/s 


In [0]:
from interval import interval
import pandas as pd
import numpy as np
import math
from numpy.linalg import inv

## Equation

### Utility

In [0]:
def width(x_i):
    return sum([x.sup - x.inf for x in x_i])

def midpoint(x_i):
    return (x_i[0].sup + x_i[0].inf) / 2

def itos(x_i):
    return f"[{x_i[0].inf:9.7f}, {x_i[0].sup:9.7f}]"

def trunc(num):
  return f"{num:9.8f}"

### Bisection method

#### Bisection with console output

In [0]:
def bisection(f, start, end, tol=0.01, iteration=1):
    X = interval[start, end]
    F = f(X)
    # No root if no zero
    if 0 not in F:
        print(iteration, "-", itos(X), itos(F), sep='\t')
        return
    # Found root if satisfies the tolerance
    if width(X) < tol:
        print(iteration, "+", itos(X), itos(F), sep='\t')
        return
    
    print(iteration, "*", itos(X), itos(F), sep='\t')
    mid = midpoint(X)
    bisection(f, start, mid, tol, iteration + 1)
    bisection(f, mid, end, tol, iteration + 1)

#### Bisection with table output

In [0]:
def bisection_tbl(f, start, end, tol=0.01):
    tbl = pd.DataFrame(columns=['Depth', 'Mark', 'Interval', 'Function extension'])

    def bisection(f, start, end, tol=0.01, iteration=1):
        X = interval[start, end]
        F = f(X)
        # No root if no zero
        if 0 not in F:
            tbl.loc[len(tbl)] = [iteration, "-", itos(X), itos(F)]
            return
        # Found root if satisfies the tolerance
        if width(X) < tol:
            tbl.loc[len(tbl)] = [iteration, "+", itos(X), itos(F)]
            return
        
        tbl.loc[len(tbl)] = [iteration, "*", itos(X), itos(F)]

        mid = midpoint(X)
        bisection(f, start, mid, tol, iteration + 1)
        bisection(f, mid, end, tol, iteration + 1)

    bisection(f, start, end, tol)
    return tbl.sort_values(['Depth', 'Interval'])

#### Example

In [0]:
func = lambda x: x**2 - 8*x + 12
bisection(func, 5.995, 6.05, 10**-2)

1	*	[5.9950000, 6.0500000]	[-0.4599750, 0.6425000]
2	*	[5.9950000, 6.0225000]	[-0.2399750, 0.3105063]
3	*	[5.9950000, 6.0087500]	[-0.1299750, 0.1450766]
4	+	[5.9950000, 6.0018750]	[-0.0749750, 0.0625035]
4	+	[6.0018750, 6.0087500]	[-0.0474965, 0.0900766]
3	*	[6.0087500, 6.0225000]	[-0.0749234, 0.2005063]
4	+	[6.0087500, 6.0156250]	[-0.0199234, 0.1177441]
4	-	[6.0156250, 6.0225000]	[0.0077441, 0.1455063]
2	*	[6.0225000, 6.0500000]	[-0.1294938, 0.4225000]
3	*	[6.0225000, 6.0362500]	[-0.0194938, 0.2563141]
4	-	[6.0225000, 6.0293750]	[0.0355062, 0.1733629]
4	-	[6.0293750, 6.0362500]	[0.0633629, 0.2013141]
3	-	[6.0362500, 6.0500000]	[0.0363141, 0.3125000]


### Moore's method

$f(x) = 0$, interval $[a, b]$

$f'(x) \in C^1[a,b]$

$F'(x)$ - inclusion monotonic interval extension of $f'(x)$

$N(X) = m(X) - \frac{f(m(X))}{F'(X)}$

Necessary conditions:  
$f(x)$ has 1 (non-multiple) root at most on $[a, b]$

#### Our pathetic try

In [0]:
def moore(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])

    X_width = width(X)
    if X_width < tol:
        print(iteration, "+", itos(X), X_width, sep='\t')
        return

    print(iteration, "*", itos(X), X_width, sep='\t')

    x_mid = midpoint(X)
    f_mid = f(x_mid)
    df_X = df(X)

    # Actually, if 0 in F'(X) then stop the algorithm with error
    # but
    if 0 in df_X:
        moore(f, df, start, x_mid, tol, iteration + 1)
        moore(f, df, x_mid,   end, tol, iteration + 1)
        return

    U = x_mid - f_mid / df(X)
    X_next = U & X

    if not X_next:
        print(iteration, "-", itos(U) + '&' + itos(X), sep='\t')
        return

    moore(f, df, X_next[0].inf, X_next[0].sup, tol, iteration + 1)

#### As in the book - very cool

In [0]:
def moore(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])
    X_width = width(X)

    # [Step 1]: Stop if no 0 in F(X)
    if 0 not in f(X):
        print(iteration, "-", itos(X), X_width, sep='\t')
        return
    
    # Return result if tolerance is satisfied
    if X_width < tol:
        print(iteration, "+", itos(X), X_width, sep='\t')
        return
    
    # [Step 2]: Half if 0 in F'(X)
    x_mid = midpoint(X)
    df_X = df(X)
    if 0 in df_X:
        print(iteration, "**", itos(X), X_width, sep='\t')
        moore(f, df, start, x_mid, tol, iteration + 1)
        moore(f, df, x_mid,   end, tol, iteration + 1)
        return
    
    # [Step 3]: Stop if X_next is empty
    f_mid = f(x_mid)
    U = x_mid - f_mid / df_X
    X_next = U & X
    if not X_next:
        print(iteration, "-", itos(U) + '&' + itos(X), sep='\t')
        return

    # [Step 4]: Continue with narrowed interval
    print(iteration, "*", itos(X), X_width, sep='\t')
    moore(f, df, X_next[0].inf, X_next[0].sup, tol, iteration + 1)

#### Example

In [0]:
func = lambda x: x**2 - 8*x + 12
dfunc = lambda x: 2*x - 8

moore(func, dfunc, 3, 7)

1	**	[3.0000000, 7.0000000]	4.0
2	**	[3.0000000, 5.0000000]	2.0
3	**	[3.0000000, 4.0000000]	1.0
4	-	[-0.1875000, 1.5312500]&[3.0000000, 3.5000000]
4	**	[3.5000000, 4.0000000]	0.5
5	-	[3.5000000, 3.7500000]	0.25
5	-	[3.7500000, 4.0000000]	0.25
3	**	[4.0000000, 5.0000000]	1.0
4	**	[4.0000000, 4.5000000]	0.5
5	-	[4.0000000, 4.2500000]	0.25
5	-	[4.2500000, 4.5000000]	0.25
4	-	[6.4687500, 8.1875000]&[4.5000000, 5.0000000]
2	*	[5.0000000, 7.0000000]	2.0
3	+	[6.0000000, 6.0000000]	0.0


### Hansen's method a.k.a. HanSolo's method

maybe move tolerance check to the end; check full width of interval with subintervals

In [0]:
def hansen(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])

    X_width = width(X)
    if X_width < tol:
        print(iteration, "+", itos(X), X_width, sep='\t')
        return

    print(iteration, "*", itos(X), X_width, sep='\t')

    x_mid = midpoint(X)
    f_mid = f(x_mid)
    df_X = df(X)

    c, d = df_X[0].inf, df_X[0].sup
    q = x_mid - f_mid / d if d != 0 else None
    p = x_mid - f_mid / c if c != 0 else None
    
    if 0 not in df_X:
        U = x_mid - f_mid / df_X
    elif f_mid == 0.0:
        print(iteration, "+", itos(X), X_width, '<====== { f == 0 }', sep='\t')
        hansen(f, df, start, x_mid, tol, iteration + 1)
        hansen(f, df, x_mid, end, tol, iteration + 1)
        return
    elif f_mid > 0.0:
        if c == 0.0:
            U = interval([-math.inf, q])
        elif d == 0.0:
            U = interval([p, math.inf])
        elif c < 0.0 and 0.0 < d:
            U = interval([-math.inf, q]) | interval([p, math.inf])
    elif f_mid < 0.0:
        if c == 0.0:
            U = interval([q, math.inf])
        elif d == 0.0:
            U = interval([-math.inf, p])
        elif c < 0.0 and 0.0 < d:
            U = interval([-math.inf, p]) | interval([q, math.inf])

    X_next = U & X

    if not X_next:
        print(iteration, 'Empty interval')
        return

    [hansen(f, df, x.inf, x.sup, tol, iteration + 1) for x in X_next]

In [0]:
func = lambda x: x**2 - 8*x + 12
dfunc = lambda x: 2*x - 8

hansen(func, dfunc, 1, 11)

1	*	[1.0000000, 11.0000000]	10.0
1	+	[1.0000000, 11.0000000]	10.0	<====== { f == 0 }
2	*	[1.0000000, 6.0000000]	5.0
3	*	[1.0000000, 2.8750000]	1.875
4	*	[1.9798177, 2.0503472]	0.07052951388888906
5	*	[1.9996689, 2.0002070]	0.0005381269403181843
6	+	[2.0000000, 2.0000000]	1.670338201087418e-08
3	*	[4.4375000, 6.0000000]	1.5625
4	*	[5.8474121, 6.0000000]	0.152587890625
5	*	[5.9985448, 6.0000000]	0.0014551915228366852
6	+	[5.9999999, 6.0000000]	1.3234889806312822e-07
2	*	[6.0000000, 11.0000000]	5.0
3	*	[6.0000000, 7.3392857]	1.3392857142857144
4	*	[6.0000000, 6.2014300]	0.201430003819711
5	*	[6.0000000, 6.0069115]	0.0069115381311943835
6	*	[6.0000000, 6.0000089]	8.925908996815224e-06
7	+	[6.0000000, 6.0000000]	1.4939161019356106e-11


### [Krawczyk] Kravchyk's method, a.k.a. Kravchenko, a.k.a Kravchuk The President

In [0]:
def kravchyk(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])
    X_width = width(X)

    # [Step 1]: Stop if no 0 in F(X)
    if 0 not in f(X):
        print(iteration, "-", itos(X), X_width, sep='\t')
        return

    # Return result if tolerance is satisfied
    if X_width < tol:
        print(iteration, "+", itos(X), X_width, sep='\t')
        return

    x_mid = midpoint(X)
    df_X = df(X)
    # Split interval if 0 in F'(X)
    if 0 in df_X:
      print(iteration, "**", itos(X), X_width, sep='\t')
      kravchyk(f, df, start, x_mid, tol, iteration + 1)
      kravchyk(f, df, x_mid,   end, tol, iteration + 1)
      return

    # Display interval if shrunk or not
    print(iteration, "*", itos(X), X_width, sep='\t')

    df_x = df(x_mid)
    K = x_mid - f(x_mid) / df_x + (1 - df(X) / df_x)*(X - x_mid)
    X_next = K & X

    if not X_next:
        print(iteration, "-", itos(K) + '&' + itos(X), sep='\t')
        return

    kravchyk(f, df, X_next[0].inf, X_next[0].sup, tol, iteration + 1)

In [0]:
func = lambda x: x**3 - 8*x**2 + 3*x + 12
dfunc = lambda x: 3*x**2 - 16*x + 3

kravchyk(func, dfunc, -2, 5)

## System of linear equations

### Moore's method

In [0]:
def print_interval(iteration, status, X, Y):
    print(iteration, status, 'X:'+itos(X), width(X), 'Y:'+itos(Y), width(Y), sep='\t')

# returns determinant, 
def det2x2(m):
    return m[0][0]*m[1][1] - m[0][1]*m[1][0]

# subinterval, t - top, b - bottom, l - left, r - right
def half_interval(x):
    mid = midpoint(x)
    return (interval[x[0].inf, mid], interval[mid, x[0].sup])

In [0]:
# Example:
# f = lambda x, y: (x + y, x - y)
# df = lambda x, y: ((1,  1),
#                    (1, -1))
# intervals = (interval[-1, 1], interval[-1, 1])
#
def moore_linear(f, df, intervals, tol=1e-6, iteration=1):
    X, Y = intervals
    X_width, Y_width = width(X), width(Y)

    # [Step 1]: Stop if no 0 in F(X, Y)
    f_XY = f(X, Y)
    if 0 not in f_XY[0] and 0 not in f_XY[1]:
        print_interval(iteration, '-', X, Y)
        return

    # Return result if tolerance is satisfied
    if max(X_width, Y_width) < tol:
        print_interval(iteration, '+', X, Y)
        return

    # [Step 2]: Half if 0 in F'(X, Y)
    x_mid, y_mid = midpoint(X), midpoint(Y)
    df_XY = df(X, Y)
    df_XY_det = det2x2(df_XY)

    if df_XY_det == 0:
        print_interval(iteration, '**', X, Y)
        Xl, Xr = half_interval(X)
        Yl, Yr = half_interval(Y)
        if X_width < tol: # half only Y
            moore_linear(f, df, (X, Yl), tol, iteration + 1)
            moore_linear(f, df, (X, Yr), tol, iteration + 1)
        elif Y_width < tol: # half only X
            moore_linear(f, df, (Xl, Y), tol, iteration + 1)
            moore_linear(f, df, (Xr, Y), tol, iteration + 1)
        else: # half everything
            moore_linear(f, df, (Xl, Yl), tol, iteration + 1)
            moore_linear(f, df, (Xl, Yr), tol, iteration + 1)
            moore_linear(f, df, (Xr, Yl), tol, iteration + 1)
            moore_linear(f, df, (Xr, Yr), tol, iteration + 1)
        return

    # [Step 3]: Stop if X_next or Y_next is empty
    f_mid = f(x_mid, y_mid)
    U_X = x_mid - ???
    U_Y = y_mid - ???
    X_next, Y_next = U_X & X, U_Y & Y
    if not X_next or not Y_next:
        print_interval(iteration, '-', X, Y)
        return

    # [Step 4]: Continue with narrowed interval
    print_interval(iteration, "*", X, Y)
    moore_linear(f, df, (X_next, Y_next), tol, iteration + 1)
    '''    
    # [Step 3]: Stop if X_next is empty
    f_mid = f(x_mid)
    U = x_mid - f_mid / df_X
    X_next = U & X
    if not X_next:
        print(iteration, "-", itos(U) + '&' + itos(X), sep='\t')
        return

    # [Step 4]: Continue with narrowed interval
    print(iteration, "*", itos(X), X_width, sep='\t')
    moore(f, df, X_next[0].inf, X_next[0].sup, tol, iteration + 1)

SyntaxError: ignored

#### Example

### Hansen's method a.k.a. HanSolo's method

In [0]:
def hansen(f_1, df_1, f_2, df_2, X_start, X_end, Y_start, Y_end, tol=1e-6, iteration=1):
    X = interval([X_start, X_end])
    Y = interval([Y_start, Y_end])

    X_width = width(X)
    Y_width = width(Y)

    x_mid = midpoint(X)
    y_mid = midpoint(Y)
    

#### Example

### Kravchyk's method, a.k.a. Kravchenko, a.k.a Kravchuk The President

##### Interval Matrix

In [0]:
class IntervalMatrix(object):
  @staticmethod
  def from_lists(arrayOfArrays):
    res = IntervalMatrix(len(arrayOfArrays), len(arrayOfArrays[0]))
    res.data = [[interval[bounds[0], bounds[1]] if isinstance(bounds, list) else interval[bounds] for bounds in row] for row in arrayOfArrays]
    return res

  @staticmethod
  def from_intervals(arrayOfArraysOfIntervals):
    res = IntervalMatrix(len(arrayOfArraysOfIntervals), len(arrayOfArraysOfIntervals[0]))
    res.data = [[i for i in row] for row in arrayOfArraysOfIntervals]
    return res
  
  def to_numpy(self):
    return np.array([[val[0].inf, val[0].sup] for row in self.data for val in row])
  
  def __init__(self, rows, columns):
    self.rows = rows
    self.columns = columns

  def __add__(self, other):
    res = IntervalMatrix(self.rows, self.columns)
    res.data = [[val1 + val2 for val1, val2 in zip(row1, row2)] for row1, row2 in zip(self.data, other.data)]
    return res

  def __sub__(self, other):
    res = IntervalMatrix(self.rows, self.columns)
    res.data = [[val1 - val2 for val1, val2 in zip(row1, row2)] for row1, row2 in zip(self.data, other.data)]
    return res

  def __mul__(self, other):
    if not self.columns == other.rows:
      raise "Nigga you blind or somethin?"
    res = IntervalMatrix(self.rows, other.columns)
    res.data = [[sum([v1 * v2 for v1,v2 in zip(leftRow, rightColumn)]) for rightColumn in other.transpose().data] for leftRow in self.data]
    return res

  def transpose(self):
    res = IntervalMatrix(self.columns, self.rows)
    res.data = [[self.data[j][i] for j in range(self.rows)] for i in range(self.columns)]
    return res

  def __getitem__(self, pos):
    row, column = pos
    return self.data[row][column]

  def __setitem__(self, pos, value):
    row, column = pos
    self.data[row][column] = value

  def __str__(self):
    res = ""
    for row in self.data:
      res += "|\t"
      for val in row:
        res += f"[{val[0].inf}; {val[0].sup}]"
      res += "\t|\n"
    return res

  def flatten(self):
    return [item for row in self.data for item in row]

  def inverse(self):
    if self.rows != self.columns:
      raise "Nigga you dumb or somethin"
    det = self[0,0]*self[1,1] - self[0,1]*self[1,0]
    res = IntervalMatrix(2,2)
    res.data[0,0] = self.data[1,1]/det
    res.data[1,1] = self.data[0,0]/det
    res.data[0,1] = -self.data[0,1]/det
    res.data[1,0] = -self.data[1,0]/det
    return res

In [0]:
def flatten(array):
  return [item for row in array for item in row]

In [0]:
def kravchyk_system(f, df, intervals, tol=1e-6, iteration=1):
    X = interval[intervals[0][0][0], intervals[0][0][1]]
    Y = interval[intervals[1][0][0], intervals[1][0][1]]

    X_width = width(X)
    Y_width = width(Y)

    F_XY = f(X, Y)
    # [Step 1]: Stop if no 0 in F(X)
    if any(0 not in f[0] for f in F_XY):
        print("F_XY", F_XY, sep='\t')
        print(iteration, "-", 
              "X", itos(X), trunc(X_width), 
              "Y", itos(Y), trunc(Y_width), 
              sep='\t')
        return

    # Return result if tolerance is satisfied
    if max(X_width, Y_width) < tol:
        print(iteration, "+", 
              "X", itos(X), trunc(X_width), 
              "Y", itos(Y), trunc(Y_width), 
              sep='\t')
        return

    x_mid = midpoint(X)
    y_mid = midpoint(Y)
    dF_XY = df(X, Y)
    
    df_xy = df(x_mid, y_mid)
    df_xy_det = df_xy[0][0]*df_xy[1][1] - df_xy[0][1]*df_xy[1][0]

    # Split interval if 0 in F'(X) or if det F'(X) = 0
    if all([0 in dF for dF in flatten(dF_XY)]) or df_xy_det == 0:
      print(iteration, "**", 
              "X", itos(X), trunc(X_width), 
              "Y", itos(Y), trunc(Y_width), 
              sep='\t')
      if X_width > Y_width:
        kravchyk_system(f, df, [[[X[0].inf, x_mid]], [intervals[1][0]]], tol, iteration + 1)
        kravchyk_system(f, df, [[[x_mid, X[0].sup]], [intervals[1][0]]], tol, iteration + 1)
      else:
        kravchyk_system(f, df, [[intervals[0][0]], [[Y[0].inf, y_mid]]], tol, iteration + 1)
        kravchyk_system(f, df, [[intervals[0][0]], [[y_mid, Y[0].sup]]], tol, iteration + 1)
      return
    e = IntervalMatrix.from_lists([[1, 0], [0, 1]])
    df_xy_inv = IntervalMatrix.from_lists(np.array([[df_xy[1][1], -df_xy[0][1]],
                          [-df_xy[1][0], df_xy[0][0]]]) / df_xy_det)
    xy_mid = IntervalMatrix.from_lists([[x_mid],[y_mid]])
    f_mid = IntervalMatrix.from_intervals(f(x_mid, y_mid))
    dF_XY = IntervalMatrix.from_intervals(dF_XY)
    XY = IntervalMatrix.from_lists(intervals)

    K = xy_mid - df_xy_inv*f_mid + (e - df_xy_inv*dF_XY)*(XY - xy_mid)
    X_next = K[0,0] & X
    Y_next = K[1,0] & Y

    if not X_next:
        print(iteration, "-", "X", itos(K[0,0]) + '&' + itos(X), sep='\t')
        return
    if not Y_next:
        print(iteration, "-", "Y", itos(K[1,0]) + '&' + itos(Y), sep='\t')
        return
    
    # Display if shrunk or not
    print(iteration, "*", 
          "X", itos(X), trunc(X_width), 
          "Y", itos(Y), trunc(Y_width), 
          sep='\t')
    
    # Force a split by the widest axis
    if X_next == X or Y_next == Y:
      if X_width > Y_width:
        kravchyk_system(f, df, [[[X[0].inf, x_mid]], [intervals[1][0]]], tol, iteration + 1)
        kravchyk_system(f, df, [[[x_mid, X[0].sup]], [intervals[1][0]]], tol, iteration + 1)
      else:
        kravchyk_system(f, df, [[intervals[0][0]], [[Y[0].inf, y_mid]]], tol, iteration + 1)
        kravchyk_system(f, df, [[intervals[0][0]], [[y_mid, Y[0].sup]]], tol, iteration + 1)
      return

    kravchyk_system(f, df, [[[X_next[0].inf, X_next[0].sup]], [[Y_next[0].inf, Y_next[0].sup]]], tol, iteration + 1)

#### Example

In [35]:
f1 = lambda x, y: x**2 + x - 3*y - 10
f2 = lambda x, y: x**2 + 3*y - 4*x + 1

f1_der_x = lambda x, y: 2*x - 3*y - 9
f1_der_y = lambda x, y: x**2 + x - 13
f2_der_x = lambda x, y: 2*x + 3*y - 3
f2_der_y = lambda x, y: x**2 - 4*x + 4

X0 = [-2, 4]
Y0 = [-4, 1]

f = lambda x, y: [[f1(x,y)], [f2(x,y)]]
df = lambda x, y: [[f1_der_x(x,y), f1_der_y(x,y)], [f2_der_x(x,y), f2_der_y(x,y)]]
intervals = [[X0], [Y0]]

kravchyk_system(f, df, intervals)

1	**	X	[-2.0000000, 4.0000000]	6.00000000	Y	[-4.0000000, 1.0000000]	5.00000000
2	*	X	[-2.0000000, 1.0000000]	3.00000000	Y	[-4.0000000, 1.0000000]	5.00000000
3	*	X	[-2.0000000, 1.0000000]	3.00000000	Y	[-4.0000000, -1.5000000]	2.50000000
4	*	X	[-2.0000000, -0.5000000]	1.50000000	Y	[-4.0000000, -1.5000000]	2.50000000
5	*	X	[-2.0000000, -0.5000000]	1.50000000	Y	[-4.0000000, -2.7500000]	1.25000000
6	*	X	[-1.9592421, -0.8029302]	1.15631196	Y	[-3.6922780, -2.9603437]	0.73193435
7	*	X	[-1.7250560, -1.1619075]	0.56314853	Y	[-3.4597285, -3.0940433]	0.36568527
F_XY	[[interval([0.005563349507550441, 0.8000863182730296])], [interval([-1.2345085964290101, -0.03696189360204283])]]
8	-	X	[-1.5401265, -1.4057853]	0.13434124	Y	[-3.2779606, -3.1898192]	0.08814142
5	*	X	[-2.0000000, -0.5000000]	1.50000000	Y	[-2.7500000, -1.5000000]	1.25000000
6	*	X	[-1.9479286, -0.7466239]	1.20130465	Y	[-2.7398718, -1.9537079]	0.78616388
7	*	X	[-1.7261368, -1.0943447]	0.63179208	Y	[-2.7322814, -2.3121895]	0.42009190
F_XY	